<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/Rspell_check%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_C_inc.csv	   df_K_inc.csv     spell_check
'Colab Notebooks'	'# df_G_inc.csv'   df_P_inc.csv
 C_ori_model.pt		 df_G_inc.csv	  'My Drive'
'답안 작성용 파일.csv'	 df_J_inc.csv	   Q_inc_model.pt


In [3]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [4]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")

## 데이터 전처리

In [5]:
df=df[(df['digit_1'] =='R')] #조건에 맞는 데이터 출력

In [6]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000022,R,91,911,당구,NaN,NaN
1,id_0000050,R,91,912,매장에서,"유치부,초등대상","레고블럭조립놀이,장소제공"
2,id_0000080,R,91,912,노래방에서,시설을 갖추고,노래방 운영
3,id_0000089,R,91,912,노래방에서,노래기기시설을 갖추고,노래연습서비스
4,id_0000144,R,90,901,사업장에서,"공연,연주",기획
...,...,...,...,...,...,...,...
29746,id_0999854,R,91,912,노래방에서,노래기기 시설을 갖추고,노래연습
29747,id_0999870,R,90,901,공연예술,기획사에서,공연기획팀
29748,id_0999886,R,91,912,노래연습장,노래연습서비스제공,노래방기기대여
29749,id_0999913,R,90,901,사업장에서,공연시설을 갖추고,공연장운영


In [7]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000022,R,91,911,당구
1,id_0000050,R,91,912,"매장에서 유치부,초등대상 레고블럭조립놀이,장소제공"
2,id_0000080,R,91,912,노래방에서 시설을 갖추고 노래방 운영
3,id_0000089,R,91,912,노래방에서 노래기기시설을 갖추고 노래연습서비스
4,id_0000144,R,90,901,"사업장에서 공연,연주 기획"
...,...,...,...,...,...
29746,id_0999854,R,91,912,노래방에서 노래기기 시설을 갖추고 노래연습
29747,id_0999870,R,90,901,공연예술 기획사에서 공연기획팀
29748,id_0999886,R,91,912,노래연습장 노래연습서비스제공 노래방기기대여
29749,id_0999913,R,90,901,사업장에서 공연시설을 갖추고 공연장운영


In [8]:
df['digit_3'].value_counts()

912    15468
911     9779
902     3039
901     1465
Name: digit_3, dtype: int64

## 맞춤법 교정

In [9]:
!pip install git+https://github.com/ssut/py-hanspell

  Cloning https://github.com/ssut/py-hanspell to /tmp/pip-req-build-9rzf3dex
  Running command git clone -q https://github.com/ssut/py-hanspell /tmp/pip-req-build-9rzf3dex
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=41b71fc0866638a8c750cbddfd5c41348a05dccb6fa204bf3868e534fd8b252c
  Stored in directory: /tmp/pip-ephem-wheel-cache-gd6nccei/wheels/31/c2/c7/ddbbd9a84ddb0a67c2e52e3324d4b7b47dcf9581af9cd784de
Successfully built py-hanspell


In [10]:
from hanspell import spell_checker

temp_result=[]
for sentence in df['text']:
  try:
    result=spell_checker.check(sentence)
    temp_result.append(result.checked)
  except:
    temp_result.append(sentence)

df['text']=pd.DataFrame(temp_result)  

In [11]:
df.to_csv('df_R_spellcheck.csv', index=False, encoding='utf-8-sig')